##### Import and read statements

In [3]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'firefox'
import json
import numpy as np
from fuzzywuzzy import process, fuzz 

In [4]:
land = pd.read_csv('land.csv')
results = pd.read_csv('results.csv')
geoj = json.load(open('ghmc-wards.geojson', 'r'))

##### Data Cleaning

In [14]:
results.head()

,Unnamed: 0,Name of the Circle,Name of the Ward,BJP,CPI,CPM,INC,AIMIM,TRS,TDP,YSRCP,Other Registered Parties with TSEC,Independents,Total Results declared,id
0,2,Kapra,ward1kapra,0,0,0,0,0,1,0,0,0,0,1,relation/7849497
1,3,Kapra,ward2drasraonagar,0,0,0,1,0,0,0,0,0,0,1,relation/7849545
4,6,Kapra,ward5mallapur,0,0,0,0,0,1,0,0,0,0,1,relation/7849614
5,7,Kapra,ward6nacharam,0,0,0,0,0,1,0,0,0,0,1,relation/7849315
6,8,Uppal,ward7chilukanagar,0,0,0,0,0,1,0,0,0,0,1,relation/7850095


In [6]:
results['Name of the Ward']=results['Name of the Ward'].apply(lambda x: str(x).replace(" -",""))
results['Name of the Ward']=results['Name of the Ward'].apply(lambda x: str(x).lower())
results['Name of the Ward']=results['Name of the Ward'].apply(lambda x: str(x).replace("."," "))

In [7]:
results['Name of the Ward']=results['Name of the Ward'].apply(lambda x: str(x).replace(" ",""))

##### Linking the two files

In [8]:
ward_id_dict = {}
for feature in geoj['features']:
    ward_id_dict[feature['properties']['name']] = feature['properties']['@id']

In [9]:
ward_id_dict2 = {}
for key in ward_id_dict.keys():
    ward_id_dict2[key.lower().replace(" ","")] = ward_id_dict[key]
ward_id_dict = ward_id_dict2 

In [10]:
results['id'] = ''
keys = ward_id_dict.keys()
for i in range(151):
    place = results['Name of the Ward'][i]
    place = str(place)
    temp = [i for i in ward_id_dict.keys()]
    temp2 = [i for i in ward_id_dict.values()]
    if place in keys:
        results['id'][i] = ward_id_dict[place]
    else:
        for j in range(len(ward_id_dict)):
            if fuzz.token_sort_ratio(place, temp[j]) >= 80:
                results['id'][i] = temp2[j]
                

/home/naren/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/naren/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
results['id'].head()
#results.to_csv('test.csv')

0    relation/7849497
1    relation/7849545
2                    
3                    
4    relation/7849614
Name: id, dtype: object

##### Further cleaning

In [12]:
results['id'].replace('',np.nan, inplace=True)
results.dropna(subset=['id'], inplace=True)

##### Finallyyyy, the plotting part!!!

In [15]:
fig = px.scatter_geo(results, geojson=geoj, color='TRS')
fig.update_geos(fitbounds="locations", visible=True)
fig.show()